# Libraries

In [ ]:
# google drive access
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/Colab Notebooks

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks


In [ ]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
import torchvision
from sklearn.metrics import accuracy_score

# Model

In [ ]:
class CCM(nn.Module):
    """
    Channel wise calibration model:
        y = x * Sigmoid(BN(Group_Conv(GAP(x))))
    """
    def __init__(self, in_channel, out_channel):
        super(CCM, self).__init__()
        self.adapool = nn.AdaptiveAvgPool2d(1)
        self.conv = nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=1, padding=0, groups=in_channel, bias=False)
        self.bn = nn.BatchNorm2d(out_channel)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        out = self.adapool(x)
        out = self.conv(out)
        out = self.bn(out)
        out = self.sig(out)
        out = x * out
        return out


class SCM(nn.Module):
    """
    Spatial Calibration Module:
        y = x + Group_Conv(x)
    """
    def __init__(self, channel):
        super(SCM, self).__init__()
        self.conv = nn.Conv2d(channel, channel, kernel_size=3,
            stride=1, padding=1, groups=channel, bias=False)

    def forward(self, x):
        out = self.conv(x)
        out = x + out
        return out


class Shortcut(nn.Module):
    """
    Modified skip connections in ResNet 
    with calibration modules.
    """
    def __init__(self, in_channel, out_channel, stride=1):
        super(Shortcut, self).__init__()
        self.conv = nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=stride, bias=False)
        self.scm = SCM(out_channel)
        self.bn = nn.BatchNorm2d(out_channel)
        self.ccm = CCM(out_channel, out_channel)

    def forward(self, x):
        out = self.conv(x)
        out = self.scm(out)
        out = self.bn(out)
        out = self.ccm(out)
        return out


class BasicBlock(nn.Module):
    expansion = 1
    """
    Convolutional blocks of ResNet with 
    skip connections and calibration modules.
    """
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        # SCM and CCM for conv1
        self.scm1 = SCM(planes)
        self.bn1 = nn.BatchNorm2d(planes)
        self.ccm1 = CCM(planes, planes)

        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        # SCM and CCM for conv2
        self.scm2 = SCM(planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.ccm2 = CCM(planes, planes)

        self.shortcut = None
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = Shortcut(in_planes, self.expansion*planes, stride)

    def forward(self, x):
        # conv1 forward pass
        out = self.conv1(x)
        out = self.scm1(out)
        out = self.bn1(out)
        out = self.ccm1(out)
        out = F.relu(out)
        # conv2 forward pass
        out = self.conv2(out)
        out = self.scm2(out)
        out = self.bn2(out)
        out = self.ccm2(out)
        if self.shortcut is not None:
            out += self.shortcut(x)
        else:
            out += x
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64
        # input convolutions
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.scm1 = SCM(64)
        self.bn1 = nn.BatchNorm2d(64)
        self.ccm1 = CCM(64, 64)
        # residual blocks
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        # classifier head
        self.classifier = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        # forward pass of input conv
        out = self.conv1(x)
        out = self.scm1(out)
        out = self.bn1(out)
        out = self.ccm1(out)
        out = F.relu(out)
        # forward pass of residual blocks
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        # forward pass of classifier head
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out


def get_resnet18(num_classes=10):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes=num_classes)

# Utils

In [ ]:
def freeze_noncalibration_layers(model):
    for child in model.children():
        # Base model convs are frozen
        # print('Child: ', type(child))
        if type(child) == nn.Conv2d:
            froze_this_layer(child, tab='')
        # Search for basic blocks in Sequentials
        elif type(child) == nn.Sequential:
            for child2 in child.children():
                # print('\t Child2: ', type(child2))
                if type(child2) == nn.Conv2d:
                    froze_this_layer(child2, tab='\t ')
                # Search for Shortcuts in Basic Blocks
                elif type(child2) == BasicBlock:
                    for child3 in child2.children():
                        # print('\t\t Child3: ', type(child3))
                        # Base model convs are frozen
                        if type(child3) == nn.Conv2d:
                            froze_this_layer(child3, tab='\t\t ')
                        elif type(child3) == Shortcut:
                            # Base model convs are frozen
                            for child4 in child3.children():
                                # print('\t\t\t Child4: ', type(child4))
                                if type(child4) == nn.Conv2d:
                                    froze_this_layer(child4, tab='\t\t\t ')

def froze_this_layer(layer, tab=''):
    for param in layer.parameters():
        # print(tab + '********* Conv2d frozen *********')
        param.requires_grad = False


def accuracy(y, y_hat):
    y_hat = np.argmax(y_hat, axis=1)
    y = np.squeeze(y)
    acc = accuracy_score(y, y_hat)
    return acc

# Dataset

In [ ]:
def get_dataloader(task_no=0, batch_size=128, subset='train', val_ratio=0.1):
    transform = torchvision.transforms.Compose(
                    [torchvision.transforms.ToTensor(),
                     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    # initial task is CIFAR10
    if task_no == 0:
        if subset == 'test':
            dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
            print('Task-{} -->  CIFAR10 ({}) loaded! Num. Samples: {}'.format(task_no, subset, len(dataset)))
            dataloader = DataLoader(dataset, batch_size=len(dataset), shuffle=False, num_workers=2)

            return dataloader
        else:
            dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
            # split train_validation
            val_len = int(len(dataset) * val_ratio)
            train_len = int(len(dataset) - val_len)
            train_data, val_data = random_split(dataset, [train_len, val_len], generator=torch.Generator().manual_seed(42))

            if subset == 'train':
                dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)
                print('Task-{} -->  CIFAR10 ({}) loaded! Num. Samples: {}'.format(task_no, subset, len(train_data)))
            elif subset == 'val':
                dataloader = DataLoader(val_data, batch_size=len(val_data), shuffle=False, num_workers=2)
                print('Task-{} -->  CIFAR10 ({}) loaded! Num. Samples: {}'.format(task_no, subset, len(val_data)))

            return dataloader
    # subsequent tasks are CIFAR100
    else:
        task_no += 2
        if subset == 'test':
            dataset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True)
            classes = list(dataset.class_to_idx.keys())
            task_classes = classes[task_no*10:10+task_no*10]
            task_class_labels = [int(dataset.class_to_idx[class_]) for class_ in task_classes]
            task_data = CustomCIFAR100(dataset, task_classes, task_class_labels, task_no=task_no, transforms=transform)
            print('Task-{} -->  CIFAR100 ({}) loaded! Num. Samples: {}'.format(task_no-2, subset, len(task_data)))
            dataloader = DataLoader(task_data, batch_size=len(task_data), shuffle=False, num_workers=2)

            return dataloader
        else:
            dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True)
            classes = list(dataset.class_to_idx.keys())
            task_classes = classes[task_no*10:10+task_no*10]
            task_class_labels = [int(dataset.class_to_idx[class_]) for class_ in task_classes]
            # split train_validation
            val_len = int(len(dataset) * val_ratio)
            train_len = int(len(dataset) - val_len)
            train_data, val_data = random_split(dataset, [train_len, val_len], generator=torch.Generator().manual_seed(42))

            if subset == 'train':
                task_data = CustomCIFAR100(train_data, task_classes, task_class_labels, task_no=task_no, transforms=transform)
                dataloader = DataLoader(task_data, batch_size=batch_size, shuffle=True, num_workers=2)
                print('Task-{} -->  CIFAR100 ({}) loaded! Num. Samples: {}'.format(task_no-2, subset, len(task_data)))
            elif subset == 'val':
                task_data = CustomCIFAR100(val_data, task_classes, task_class_labels, task_no=task_no, transforms=transform)
                dataloader = DataLoader(task_data, batch_size=len(task_data), shuffle=False, num_workers=2)
                print('Task-{} -->  CIFAR100 ({}) loaded! Num. Samples: {}'.format(task_no-2, subset, len(task_data)))
            
            return dataloader


class CustomCIFAR100(Dataset):
    def __init__(self, dataset, class_names, class_idxs, task_no=0, transforms=None):
        super(CustomCIFAR100, self).__init__()
        self.dataset = dataset
        self.task_no = task_no
        self.class_names = class_names
        self.class_idxs = class_idxs
        self.transforms = transforms
        self.idxs = [idx for idx in range(len(dataset)) if (self.dataset[idx][1] in self.class_idxs)]
        self.labels = [self.dataset[idx][1] - 10*self.task_no for idx in self.idxs]

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img, _ = self.dataset[self.idxs[idx]]
        y = self.labels[idx]

        # convert labels to torch tensor too
        if not isinstance(y, np.ndarray):
            y = np.array(y)
            y = torch.from_numpy(y)

        # apply input image transformations
        if self.transforms:
            img = self.transforms(img)
        return img, y

In [ ]:
"""
train_loader0 = get_dataloader(task_no=0, batch_size=128, subset='train')
train_loader4 = get_dataloader(task_no=4, batch_size=64, subset='train')

val_loader0 = get_dataloader(task_no=0, subset='val')
val_loader4 = get_dataloader(task_no=4, subset='val')

test_loader0 = get_dataloader(task_no=0, subset='test')
test_loader4 = get_dataloader(task_no=4, subset='test')
"""

"\ntrain_loader0 = get_dataloader(task_no=0, batch_size=128, subset='train')\ntrain_loader4 = get_dataloader(task_no=4, batch_size=64, subset='train')\n\nval_loader0 = get_dataloader(task_no=0, subset='val')\nval_loader4 = get_dataloader(task_no=4, subset='val')\n\ntest_loader0 = get_dataloader(task_no=0, subset='test')\ntest_loader4 = get_dataloader(task_no=4, subset='test')\n"

# Main

In [ ]:
if torch.cuda.is_available():
  print("Cuda (GPU support) is available and enabled!")
  device = torch.device("cuda")
else:
  print("Cuda (GPU support) is not available")
  device = torch.device("cpu")

Cuda (GPU support) is available and enabled!


In [ ]:
def train(verbose=True):
    # create ResNet18 model
    model = ResNet(BasicBlock, [2,2,2,2], num_classes=10)
    model = model.to(device)

    # tensorboard logger
    writer = SummaryWriter(log_dir='runs/splitCIFAR_experiment')
    model_path = './models/splitCIFAR_experiment/'

    for task_no in range(6):
        task_tag = 'task_' + str(task_no)
        
        # training parameters
        if task_no == 0:
            batch_size = 64
            lr = 1e-2
            epochs = 15
        else:
            batch_size = 64
            lr = 1e-2
            epochs = 50

        # freeze layers other than CCM and SCM,
        # add new classifier head for next task
        if task_no > 0:
            print('************* Re-calibrating model parameters *************')
            model = model.to('cpu')
            freeze_noncalibration_layers(model)
            model.classifier = nn.Linear(512*BasicBlock.expansion, 10)
            model = model.to(device)

        # check layer freezing
        num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print('************* Task: {}, Number of Trainable Parameters: {} *************'.format(task_no, num_trainable_params))

        # get dataset for task no
        task_train_loader = get_dataloader(task_no=task_no, subset='train', batch_size=batch_size)
        task_val_loader = get_dataloader(task_no=task_no, subset='val')

        # loss, optimizer and lr-scheduler are identical for each task
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr, momentum=0.9)
        if task_no == 0:
            scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10], gamma=0.1)
        else:
            scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15, 30], gamma=0.1)

        # training iterations
        tr_loss_hist, val_loss_hist, val_acc_hist = [], [], []
        num_iter = 0
        for e in range(epochs):
            # put model to training mode after each eval
            model.train()
            # training for an epoch
            tqdm_desc = 'Epoch ' + str(e+1) + '/' + str(epochs)
            for _, data in tqdm(enumerate(task_train_loader), ascii=True, desc=tqdm_desc):
                # get the data
                x, y = data
                x, y = x.to(device), y.to(device)

                # clear gradients
                optimizer.zero_grad()

                # calculate loss
                yhat = model(x)
                loss = criterion(yhat, y)

                # backprop
                loss.backward()

                # update parameters
                optimizer.step()

                # store training loss
                tr_loss_hist.append(loss.item())
                # save training loss per minibatch iterations
                writer.add_scalar('tr_loss/'+task_tag, loss.item(), num_iter)
                num_iter += 1

            # get average training loss for last 5 iterations
            tr_loss = np.sum(tr_loss_hist[:-6:-1])/5
            # update learning-rate
            scheduler.step()
            
            # validation after each epoch
            model.eval()
            with torch.no_grad():
                for i, data in enumerate(task_val_loader):
                    x, y = data
                    x, y = x.to(device), y.to(device)
                    yhat = model(x)
                # compute validation metrics
                val_loss = criterion(yhat, y) .item()
                val_acc = accuracy(y.to('cpu').numpy(), yhat.to('cpu').numpy())

            # save validation scores
            val_loss_hist.append(val_loss)
            val_acc_hist.append(val_acc)
            # save to tensorboard
            writer.add_scalar('val_loss/'+task_tag, val_loss, e)
            writer.add_scalar('val_acc/'+task_tag, val_acc, e)
            # display training info
            if verbose:
                print('tr_loss:{} || val_loss:{} || val_acc:{}'.format(tr_loss, val_loss, val_acc))
                
        # save the model for current task 
        print('************* Saving model for Task-{} *************'.format(task_no))
        model_filename = 'splitcifar_model_task_' + str(task_no) + '.pt'
        torch.save(model, model_path + model_filename)
        print('************* End of Task-{} training *************'.format(task_no))

In [ ]:
train()

************* Task: 0, Number of Trainable Parameters: 11231562 *************
Files already downloaded and verified
Task-0 -->  CIFAR10 (train) loaded! Num. Samples: 45000
Files already downloaded and verified
Task-0 -->  CIFAR10 (val) loaded! Num. Samples: 5000


Epoch 1/15: 704it [01:17,  9.07it/s]


tr_loss:1.3788735151290894 || val_loss:1.2268131971359253 || val_acc:0.5414


Epoch 2/15: 704it [01:19,  8.83it/s]


tr_loss:0.9486603379249573 || val_loss:0.9298802614212036 || val_acc:0.671


...

tr_loss:0.5681861698627472 || val_loss:0.7208508849143982 || val_acc:0.7586206896551724


Epoch 49/50: 71it [00:05, 12.88it/s]


tr_loss:0.565522301197052 || val_loss:0.7175218462944031 || val_acc:0.7626774847870182


Epoch 50/50: 71it [00:05, 12.80it/s]


tr_loss:0.5736058533191681 || val_loss:0.7175626754760742 || val_acc:0.7586206896551724
************* Saving model for Task-5 *************
************* End of Task-5 training *************


In [ ]:
def test():
    model_path = './models/splitCIFAR_experiment/'
    for task_no in range(6):
        # load models per task
        series_num = 'splitcifar_model_task_' + str(task_no) + '.pt'
        model = torch.load(model_path + series_num)
        model = model.to(device)
        model.eval()
        # get task specific test data
        test_loader = get_dataloader(subset='test', task_no=task_no)
        criterion = nn.CrossEntropyLoss()

        with torch.no_grad():
            for _, data in enumerate(test_loader):
                x, y = data
                x, y = x.to(device), y.to(device)
                yhat = model(x)

                test_loss = criterion(yhat, y).item()
                test_acc = accuracy(y.to('cpu').numpy(), yhat.to('cpu').numpy())
            print('Task-{} || test_loss:{} || test_acc:{}'.format(task_no, test_loss, test_acc))

In [ ]:
test()

Files already downloaded and verified
Task-0 -->  CIFAR10 (test) loaded! Num. Samples: 10000
Task-0 || test_loss:0.6690012216567993 || test_acc:0.8379
Files already downloaded and verified
Task-1 -->  CIFAR100 (test) loaded! Num. Samples: 1000
Task-1 || test_loss:0.8082564473152161 || test_acc:0.74
Files already downloaded and verified
Task-2 -->  CIFAR100 (test) loaded! Num. Samples: 1000
Task-2 || test_loss:0.6675814986228943 || test_acc:0.784
Files already downloaded and verified
Task-3 -->  CIFAR100 (test) loaded! Num. Samples: 1000
Task-3 || test_loss:0.7235320806503296 || test_acc:0.726
Files already downloaded and verified
Task-4 -->  CIFAR100 (test) loaded! Num. Samples: 1000
Task-4 || test_loss:0.7428212761878967 || test_acc:0.756
Files already downloaded and verified
Task-5 -->  CIFAR100 (test) loaded! Num. Samples: 1000
Task-5 || test_loss:0.7409536242485046 || test_acc:0.751
